In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

Text Vectorization

In [ ]:
import jieba
dd=pd.DataFrame(df['text'].astype(str))
words = []
for i, row in dd.iterrows():
    word = jieba.cut(row['text'])
    result = ' '.join(word) 
    words.append(result)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X = vect.fit_transform(words)
X = X.toarray()

PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.9)
pca.fit(X)
X1=pca.transform(X)
X1.shape

In [ ]:
X=X1
y=df['score']
from collections import Counter
Counter(y)

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=933)
X_n, y_n=smote.fit_resample(X,y)
Counter(y_n)

In [ ]:
XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.2, random_state=123)

In [ ]:
from xgboost import XGBClassifier
mode = XGBClassifier()
model = mode.fit(X=X_train, y=y_train)

In [ ]:
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
score

Confusion Matrix

In [ ]:
def conf_matrix(y, y_pred, title, labels):
    fig, ax =plt.subplots(figsize=(7.5,7.5))
    ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Purples", fmt='g', cbar=False, annot_kws={"size":30})
    plt.title(title, fontsize=25)
    ax.xaxis.set_ticklabels(labels, fontsize=16) 
    ax.yaxis.set_ticklabels(labels, fontsize=14.5)
    ax.set_ylabel('Test', fontsize=25)
    ax.set_xlabel('Predicted', fontsize=25)
    plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
sentiments = ["1","2","3","4","5"]
print('Classification Report for Naive Bayes:\n',classification_report(y_test, y_pred, target_names=sentiments))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
conf_matrix(y_test,y_pred,'XGBoost after PCA and Oversampling\nConfusion Matrix', sentiments)
plt.savefig('D:/desktop/hxjzxgb.jpg',dpi=800,bbox_inches='tight')

Multi-class ROC Curve

In [ ]:
from sklearn.preprocessing import label_binarize
Y_test1 = label_binarize(y_test,classes=[1, 2, 3, 4, 5])
n_classes = Y_test1.shape[1] 
Y_train1 = label_binarize(y_train, classes=[1, 2, 3, 4, 5])

In [ ]:
from xgboost import XGBClassifier
clt = OneVsRestClassifier(XGBClassifier(eval_metric=['logloss']))
clf = clt.fit(X=X_train, y=Y_train1)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
y_score = clf.predict_proba(X_test)
# (1) For each class
precision = dict()
recall = dict()
average_precision = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y_test1[:, i],
                                                        y_score[:, i])

    average_precision[i] = average_precision_score(Y_test1[:, i], 
                                                   y_score[:, i])


# (2) A "macro-average": quantifying score on all classes jointly
precision["macro"], recall["macro"], _ = precision_recall_curve(Y_test1.ravel(),
                                                                y_score.ravel())

average_precision["macro"] = average_precision_score(Y_test1, y_score,
                                                     average="macro")

print('Average precision score, macro-averaged over all classes: {0:0.2f}'.format(average_precision["macro"]))

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.step(recall['macro'], precision['macro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()

In [ ]:
from itertools import cycle
from scipy import interp
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(Y_test1[:, i], y_score[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    
fpr["micro"], tpr["micro"], _ = metrics.roc_curve(Y_test1.ravel(), y_score.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()